In [ ]:
import numpy as np, pandas as pd, cvxpy as cp
from scipy.linalg import block_diag

In [35]:
# ----------------- Load and Prepare Data -------------------
data = pd.read_csv("3DailyReturns.csv", index_col=0, usecols=range(1, 9))
n_assets = data.shape[1]
rows_per_qtr = data.shape[0] // 4
quarters = [data.iloc[i*rows_per_qtr:(i+1)*rows_per_qtr] for i in range(4)]
excess_returns = [q - 1.0 for q in quarters]  # assume rf = 0

mu_q1 = excess_returns[0].mean().values * rows_per_qtr
cov_q1 = excess_returns[0].cov().values * rows_per_qtr

# ----------------- Strategy Solvers -------------------
def stack_mean_cov(mu, cov, H):
    μ = [mu * (H - k) for k in range(H)]
    Σ = [cov * (H - k) for k in range(H)]
    return np.concatenate(μ), block_diag(*Σ)

def solve_multi_period(mu, cov, H, target):
    μvec, Q = stack_mean_cov(mu, cov, H)
    w = cp.Variable(Q.shape[0])
    cons = [cp.sum(w[:n_assets]) == 1]
    for k in range(1, H):
        cons.append(cp.sum(w[k*n_assets:(k+1)*n_assets]) == 0)
    cons.append(μvec @ w == target)
    cp.Problem(cp.Minimize(cp.quad_form(w, Q)), cons).solve()
    return w.value

def solve_spo(mu, cov, target, H):
    mu_H = mu * H
    cov_H = cov * H
    w = cp.Variable(n_assets)
    prob = cp.Problem(cp.Minimize(cp.quad_form(w, cov_H)),
                      [cp.sum(w) == 1, mu_H @ w == target])
    prob.solve()
    return w.value

# ----------------- Quarter Execution -------------------
def apply_quarter(w_prev, trade, gross_q):
    gross_vec = gross_q.prod().values
    w_post = w_prev + trade
    growth = (w_post * gross_vec).sum()
    w_new = (w_post * gross_vec) / growth
    return w_new, growth - 1.0

# ----------------- Backtest Strategies -------------------
def backtest(strategy, print_weights=True):
    w = {
        "OpenLoop": w_ol.copy(),
        "MPC": w_ol.copy(),
        "SPO": w_spo.copy(),
        "EqualWeight": np.ones(n_assets) / n_assets
    }[strategy]
    returns = []

    for k in range(1, 4):
        q = quarters[k]

        if print_weights:
            print(f"{strategy} - Start of Quarter {k}: Weights = {np.round(w, 4)}")

        if strategy == "OpenLoop":
            trade = np.zeros(n_assets) if k == 1 else u_ol[k-2]
        elif strategy == "MPC":
            mu_k = excess_returns[k-1].mean().values * rows_per_qtr
            cov_k = excess_returns[k-1].cov().values * rows_per_qtr
            w_plan = solve_multi_period(mu_k, cov_k, 4 - k, target)
            trade = w_plan[:n_assets] - w
        else:
            trade = np.zeros(n_assets)

        w, r = apply_quarter(w, trade, q)
        returns.append(r)

    total = np.prod([1 + r for r in returns]) - 1
    vol = np.std(returns, ddof=1) * np.sqrt(len(returns))
    return total, vol, returns

# ----------------- Run All Strategies -------------------
H = 3
target = 0.3221

# Open-loop setup
w_plan = solve_multi_period(mu_q1, cov_q1, H, target)
w_ol = w_plan[:n_assets]
u_ol = [w_plan[(i+1)*n_assets:(i+2)*n_assets] for i in range(H-1)]

# SPO setup
w_spo = solve_spo(mu_q1, cov_q1, target, H)

results = {
    "OpenLoop": backtest("OpenLoop"),
    "MPC": backtest("MPC"),
    "SPO": backtest("SPO"),
    "EqualWeight": backtest("EqualWeight")
}

# ----------------- Print Results -------------------
print(f"{'Strategy':12} | {'Quarterly Returns':30} | {'Total':>6} | {'Annσ':>6}")
print("-" * 70)
for name, (tot, vol, r_list) in results.items():
    print(f"{name:12} | {np.round(r_list,4)} | {tot:6.4f} | {vol:6.4f}")


OSError: [Errno 22] Invalid argument: '3DailyReturns.csv'

In [34]:
# ----------------- Sharpe Ratio Calculation -------------------
risk_free_rate = 0.0472

def sharpe_ratio(total_return, volatility):
    excess_ret = total_return - risk_free_rate
    return excess_ret / volatility

# ----------------- Extract key values -------------------
ret_ol, vol_ol, _ = results["OpenLoop"]
ret_mpc, vol_mpc, _ = results["MPC"]
ret_spo, vol_spo, _ = results["SPO"]
ret_eq, vol_eq, _ = results["EqualWeight"]

sharpe_ol = sharpe_ratio(ret_ol, vol_ol)
sharpe_mpc = sharpe_ratio(ret_mpc, vol_mpc)
sharpe_spo = sharpe_ratio(ret_spo, vol_spo)
sharpe_eq = sharpe_ratio(ret_eq, vol_eq)

# ----------------- Individual Results -------------------
print("\n============== MULTI-PERIOD OPEN-LOOP (OL) ==============")
print(f"Scenario return : {ret_ol:.4f}")
print(f"Std Dev         : {vol_ol:.4f}")

print("\n============== MODEL PREDICTIVE CONTROL (MPC) ===========")
print(f"Scenario return : {ret_mpc:.4f}")
print(f"Std Dev         : {vol_mpc:.4f}")

print("\n============== SINGLE-PERIOD OPTIMISATION (SPO) =========")
print(f"Scenario return : {ret_spo:.4f}")
print(f"Std Dev         : {vol_spo:.4f}")

# ----------------- Aligned Summary Table -------------------
print("\n==============    SUMMARY    ======================")
print(f"{'Model':<35} {'Return':>10} {'Std Dev':>10} {'Sharpe':>10}")
print("-" * 65)
print(f"{'Multi-period Open-loop (OL)':<35} {ret_ol:10.4f} {vol_ol:10.4f} {sharpe_ol:10.4f}")
print(f"{'Model Predictive Control (MPC)':<35} {ret_mpc:10.4f} {vol_mpc:10.4f} {sharpe_mpc:10.4f}")
print(f"{'Single-period Optimisation (SPO)':<35} {ret_spo:10.4f} {vol_spo:10.4f} {sharpe_spo:10.4f}")
print("-" * 65)
print(f"{'Equal-weighted':<35} {ret_eq:10.4f} {vol_eq:10.4f} {sharpe_eq:10.4f}")


============== MULTI-PERIOD OPEN-LOOP (OL) ==============
Scenario return : 0.0525
Std Dev         : 0.0668

============== MODEL PREDICTIVE CONTROL (MPC) ===========
Scenario return : 0.1230
Std Dev         : 0.0488

============== SINGLE-PERIOD OPTIMISATION (SPO) =========
Scenario return : 0.1269
Std Dev         : 0.0507

==============    SUMMARY    ======================
Model                                   Return    Std Dev     Sharpe
-----------------------------------------------------------------
Multi-period Open-loop (OL)             0.0525     0.0668     0.0797
Model Predictive Control (MPC)          0.1230     0.0488     1.5535
Single-period Optimisation (SPO)        0.1269     0.0507     1.5725
-----------------------------------------------------------------
Equal-weighted                          0.3221     0.0621     4.4265
